In [1]:
import numpy as np
import deepchem as dc
from source.utils.npz_utils import get_smiles_and_filepaths_from_valid_npz
from typing import List

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (/home/nobilm@usi.ch/miniconda3/envs/rdkit_pyg_geqm9_clone/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


In [2]:
def scaffold_splitter(smile_list:List[str], save_dir, filepath_list:List[str]=None, frac_train: float = 0.8, frac_valid: float = 0.1, frac_test: float = 0.1, seed: int=42):
    '''https://deepchem.readthedocs.io/en/latest/api_reference/splitters.html#scaffoldsplitter
    https://deepchem.readthedocs.io/en/latest/api_reference/data.html#deepchem.data.DiskDataset.from_numpy

    save_dir – The directory to write this dataset to. If 'tmp' is specified, will use a default temporary directory instead.
    '''
    if save_dir == 'tmp':
        save_dir = None

    _ = np.zeros(len(smile_list))
    # creation of a deepchem dataset with the smile codes in the ids field
    if filepath_list:
        dataset = dc.data.DiskDataset.from_numpy(X=filepath_list,y=_,w=_,ids=smile_list, data_dir=save_dir)
    else:
        dataset = dc.data.DiskDataset.from_numpy(X=_,y=_,w=_,ids=smile_list, data_dir=save_dir)

    scaffoldsplitter = dc.splits.ScaffoldSplitter()
    train_idxs, val_idxs, test_idxs = scaffoldsplitter.split(dataset, frac_train, frac_valid, frac_test, seed)

    # ugly but avoids N bool condition eval, keep same out List[Dict] format
    if filepath_list:
        train_smiles = [{'filepath':dataset.X[i], 'smiles':smile_list[i]} for i in train_idxs]
        val_smiles = [{'filepath':dataset.X[i], 'smiles':smile_list[i]} for i in val_idxs]
        test_smiles = [{'filepath':dataset.X[i], 'smiles':smile_list[i]} for i in test_idxs]
    else:
        train_smiles = [{'smiles':smile_list[i]} for i in train_idxs]
        val_smiles = [{'smiles':smile_list[i]} for i in val_idxs]
        test_smiles = [{'smiles':smile_list[i]} for i in test_idxs]

    return train_smiles,val_smiles,test_smiles


In [3]:
dir = '/storage_common/nobilm/pretrain_paper/guacamol/5k'
smiles, filepaths = get_smiles_and_filepaths_from_valid_npz(dir)

In [4]:
train_smiles,val_smiles,test_smiles=scaffold_splitter(smile_list=smiles, save_dir='tmp', filepath_list=filepaths)

In [5]:
len(train_smiles), train_smiles

(1160,
 [{'filepath': '/storage_common/nobilm/pretrain_paper/guacamol/5k/mol_1032705.npz',
   'smiles': 'C=C(C#N)C(OC(C)=O)c1ccc([N+](=O)[O-])cc1'},
  {'filepath': '/storage_common/nobilm/pretrain_paper/guacamol/5k/mol_1337040.npz',
   'smiles': 'CC(=O)Nc1ccc(S(=O)(=O)N=S(C)C)cc1'},
  {'filepath': '/storage_common/nobilm/pretrain_paper/guacamol/5k/mol_694976.npz',
   'smiles': 'N#CC(C#N)=Cc1ccc(O)cc1'},
  {'filepath': '/storage_common/nobilm/pretrain_paper/guacamol/5k/mol_164822.npz',
   'smiles': 'CC(C)CNC(=O)NC(=O)COc1ccc(C=O)cc1'},
  {'filepath': '/storage_common/nobilm/pretrain_paper/guacamol/5k/mol_378178.npz',
   'smiles': 'CCCCCCCCCCC(=O)NCCc1ccc(OC)c(OC)c1'},
  {'filepath': '/storage_common/nobilm/pretrain_paper/guacamol/5k/mol_1138141.npz',
   'smiles': 'C=CCc1cc(Cl)ccc1OCCOCCNCCOC'},
  {'filepath': '/storage_common/nobilm/pretrain_paper/guacamol/5k/mol_931122.npz',
   'smiles': 'CCCCCC=CCC=CCCCCCCCC(=O)Nc1c(OC)cc(OC)cc1OC'},
  {'filepath': '/storage_common/nobilm/pretrain_pap

In [6]:
len(val_smiles), val_smiles

(145,
 [{'filepath': '/storage_common/nobilm/pretrain_paper/guacamol/5k/mol_1412758.npz',
   'smiles': 'N#Cc1nc(C(=O)NCCCCC(=O)O)c(O)c2ccn(Cc3ccccc3)c(=O)c12'},
  {'filepath': '/storage_common/nobilm/pretrain_paper/guacamol/5k/mol_48227.npz',
   'smiles': 'O=C(O)c1cc(-c2nc(-c3ccc(Cl)c(Cl)c3)cs2)ccc1-c1cccc([N+](=O)[O-])c1'},
  {'filepath': '/storage_common/nobilm/pretrain_paper/guacamol/5k/mol_74354.npz',
   'smiles': 'COc1cccc(NC(=O)C(Sc2nnc(Nc3ccccc3C)s2)c2ccccc2)c1'},
  {'filepath': '/storage_common/nobilm/pretrain_paper/guacamol/5k/mol_167692.npz',
   'smiles': 'COCC1CC2(CCN(Cc3cccnc3)CC2)CN1C(C)C'},
  {'filepath': '/storage_common/nobilm/pretrain_paper/guacamol/5k/mol_143415.npz',
   'smiles': 'CN(C)c1cccc2c(S(=O)(=O)NC(CCCN=C(N)N)C(=O)NC3CCCCC3)cccc12'},
  {'filepath': '/storage_common/nobilm/pretrain_paper/guacamol/5k/mol_924594.npz',
   'smiles': 'COc1ccc(C(CC(=O)N2CCN(Cc3nc4ccc(C)cc4o3)CC2)c2ccccc2)cc1'},
  {'filepath': '/storage_common/nobilm/pretrain_paper/guacamol/5k/mol_62

In [7]:
len(test_smiles), test_smiles

(145,
 [{'filepath': '/storage_common/nobilm/pretrain_paper/guacamol/5k/mol_1383431.npz',
   'smiles': 'C#Cc1cccc(C(=O)NCc2nc(-c3ccccc3NC(=O)C3CCCO3)oc2C)c1'},
  {'filepath': '/storage_common/nobilm/pretrain_paper/guacamol/5k/mol_424221.npz',
   'smiles': 'CC1CN=C(Nc2ccccc2)N1CC1CCCCC1'},
  {'filepath': '/storage_common/nobilm/pretrain_paper/guacamol/5k/mol_138237.npz',
   'smiles': 'O=C(CCCCCNc1c2c(nc3cc(Cl)ccc13)CCCC2)NCCc1c[nH]c2ccccc12'},
  {'filepath': '/storage_common/nobilm/pretrain_paper/guacamol/5k/mol_212993.npz',
   'smiles': 'COc1cc(CNC(=O)C=Cc2ccc(NC=O)cc2)ccc1O'},
  {'filepath': '/storage_common/nobilm/pretrain_paper/guacamol/5k/mol_1449533.npz',
   'smiles': 'O=C1NC(=O)C(C=Nc2ccc([N+](=O)[O-])cc2)C(=O)N1'},
  {'filepath': '/storage_common/nobilm/pretrain_paper/guacamol/5k/mol_805573.npz',
   'smiles': 'CC(C)CC(NC(=O)CCSc1ccc2ccccc2c1)C(=O)NC1CC(=O)OC1O'},
  {'filepath': '/storage_common/nobilm/pretrain_paper/guacamol/5k/mol_1110734.npz',
   'smiles': 'O=C1C=C(c2cccc(Br)c